In [1]:
#Import Modules
import numpy as np
import pandas as pd
import scipy
import shap
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

c:\Users\grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Load Data
admission_annotation = pd.read_csv("../Key data files/admission_annotation.csv").rename(columns = {'X':'patient_ids'})
admission_norm_gene_exp = pd.read_csv("../Key data files/admission_norm_gene_exp_df.csv").rename(columns = {'Unnamed: 0':'gene_ids'})
delirium_cohort_demographics = pd.read_excel("../Key data files\delirium cohort demographics.xlsx").iloc[:128,:]
gene_symbols = pd.read_csv("../Key data files\gene_symbols.csv").iloc[:,1:]
serial_norm_gene_exp = pd.read_csv("../Key data files\serial_norm_gene_exp_df.csv").rename(columns = {'Unnamed: 0':'gene_symbols'})
serial_samples_annotation = pd.read_csv("../Key data files\serial_samples_annotation.csv").rename(columns = {'X':'patient_ids'})

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\g'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\g'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\grace\AppData\Local\Temp\ipykernel_14788\4287026794.py:4: SyntaxWarning: invalid escape sequence '\d'
  delirium_cohort_demographics = pd.read_excel("../Key data files\delirium cohort demographics.xlsx").iloc[:128,:]
C:\Users\grace\AppData\Local\Temp\ipykernel_14788\4287026794.py:5: SyntaxWarning: invalid escape sequence '\g'
  gene_symbols = pd.read_csv("../Key data files\gene_symbols.csv").iloc[:,1:]
C:\Users\grace\AppData\Local\Temp\ipykernel_14788\4287026794.py:6: SyntaxWarning: invalid escape sequence '\s'
  serial_norm_gene_exp = pd.read_csv("../Key data files\serial_norm_gene_e

In [3]:
delirium_cohort_demographics.columns

Index(['Master Record ID', 'Event Name',
       'Sequential Organ Failure Assessment (SOFA) Score', 'Age at Admission',
       'Sex at Birth', 'Race', 'Hispanic ethnicity',
       'Length of Hospitalization', 'ICU Length of Stay', 'SOFA', 'WHO Scale',
       'Remdesivir', 'Remdesivir Date Started (HD)',
       'Remdesiver date ended (HD)', 'Convalescent plasma',
       'Conv Plasma Date Started', 'Conv Plasma HD ended', 'IV steroids',
       'IV steroid HD started', 'IV steroid HD ended', 'Dex given',
       'Methylpred given', 'Hydrocort given', 'Dementia', 'Deceased',
       'Cause of death',
       'Was patient on ECMO at any point since the last study visit?',
       'Non-invasive ventilation (e.g. BiPAP, CPAP)  ',
       'Nasal cannula, face mask, or HFNC oxygen therapy',
       'Maximum O2 flow via NC or face mask', 'Invasive ventilation',
       'Delirium at any time during hospitalization'],
      dtype='object')

In [4]:
cleaned_delirium_demographics = delirium_cohort_demographics[['Race','SOFA','WHO Scale','Age at Admission','Sex at Birth','Dementia','Delirium at any time during hospitalization']]
cleaned_delirium_demographics.head()

,Race,SOFA,WHO Scale,Age at Admission,Sex at Birth,Dementia,Delirium at any time during hospitalization
0,White,10,7.0,34.8,Female,No,1.0
1,Other / Multiple Races,4,5.0,43.6,Female,No,0.0
2,Native Hawaiian / Other Pacific Islander,0,3.0,83.4,Male,Yes,1.0
3,White,9,7.0,67,Male,No,0.0
4,Black / African American,10,7.0,50.7,Female,No,0.0


In [5]:
cleaned_delirium_demographics = cleaned_delirium_demographics.drop(index=22).dropna()

In [6]:
cleaned_delirium_demographics

,Race,SOFA,WHO Scale,Age at Admission,Sex at Birth,Dementia,Delirium at any time during hospitalization
0,White,10,7.0,34.8,Female,No,1.0
1,Other / Multiple Races,4,5.0,43.6,Female,No,0.0
2,Native Hawaiian / Other Pacific Islander,0,3.0,83.4,Male,Yes,1.0
3,White,9,7.0,67,Male,No,0.0
4,Black / African American,10,7.0,50.7,Female,No,0.0
...,...,...,...,...,...,...,...
122,Other / Multiple Races,20,7.0,56.8,Male,No,1.0
123,Other / Multiple Races,0,4.0,55,Male,No,0.0
124,Other / Multiple Races,1,5.0,68.3,Male,No,1.0
125,White,10,7.0,66.1,Male,No,0.0


In [7]:
X = cleaned_delirium_demographics.iloc[:,:-1]
Y = cleaned_delirium_demographics.iloc[:,-1]
X

,Race,SOFA,WHO Scale,Age at Admission,Sex at Birth,Dementia
0,White,10,7.0,34.8,Female,No
1,Other / Multiple Races,4,5.0,43.6,Female,No
2,Native Hawaiian / Other Pacific Islander,0,3.0,83.4,Male,Yes
3,White,9,7.0,67,Male,No
4,Black / African American,10,7.0,50.7,Female,No
...,...,...,...,...,...,...
122,Other / Multiple Races,20,7.0,56.8,Male,No
123,Other / Multiple Races,0,4.0,55,Male,No
124,Other / Multiple Races,1,5.0,68.3,Male,No
125,White,10,7.0,66.1,Male,No


In [8]:
#X = X.drop(X[X['Dementia'] == 'Dementia'].index)
X

,Race,SOFA,WHO Scale,Age at Admission,Sex at Birth,Dementia
0,White,10,7.0,34.8,Female,No
1,Other / Multiple Races,4,5.0,43.6,Female,No
2,Native Hawaiian / Other Pacific Islander,0,3.0,83.4,Male,Yes
3,White,9,7.0,67,Male,No
4,Black / African American,10,7.0,50.7,Female,No
...,...,...,...,...,...,...
122,Other / Multiple Races,20,7.0,56.8,Male,No
123,Other / Multiple Races,0,4.0,55,Male,No
124,Other / Multiple Races,1,5.0,68.3,Male,No
125,White,10,7.0,66.1,Male,No


In [9]:
def ohe_column(data, column_name):
    """
    One-hot-encodes roof material. New columns are of the form "Roof Material_MATERIAL".
    """

    #data = data.drop(data[data[column_name] == 'unknown'].index)

    ohe = OneHotEncoder()
    ohe.fit(data[[column_name]])

    new_data = ohe.transform(data[[column_name]]).toarray()
    new_df = pd.DataFrame(data = new_data, columns = ohe.get_feature_names_out(), index = data.index)
    df = data.join(new_df).drop(columns = [column_name])
    return df

def ohe_columns(data, column_names):
    """
    One-hot-encodes roof material. New columns are of the form "Roof Material_MATERIAL".
    """
    for column_name in column_names:
        data = ohe_column(data, column_name)
    return data

In [10]:
ohe_c = ['Sex at Birth','Race','Dementia']
X= ohe_columns(X,ohe_c)

In [13]:
X.to_csv("demographs.csv")

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

In [ ]:
lr = LogisticRegression(fit_intercept=True, penalty='l2')

lr.fit(X_train, Y_train)
lr.intercept_, lr.coef_

In [ ]:
train_accuracy = np.mean(lr.predict(X_train)==Y_train)
test_accuracy = np.mean(lr.predict(X_test)==Y_test)

print(f"Train accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, lr.predict(X_test)) 
roc_auc = auc(fpr, tpr)
# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Delirium Classification')
plt.legend()
plt.show()

In [ ]:

# Train Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, Y_train)


In [ ]:
# Create SHAP KernelExplainer for Logistic Regression
explainer = shap.KernelExplainer(lambda x: lr_model.predict_proba(x)[:, 1], X_train)  # Focus on the positive class (1)
shap_values = explainer.shap_values(X_train)


In [ ]:

# Detailed SHAP summary plot
shap.summary_plot(shap_values, X_train, show=False)
plt.title("Logistic Regression SHAP Summary Plot")
plt.show()

In [ ]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)

In [ ]:
# Create LightGBM dataset
train_data = lgb.Dataset(X_train, label=Y_train)
test_data = lgb.Dataset(X_test, label=Y_test, reference=train_data)

# Set parameters
params = {
    'objective': 'binary',  # Use 'multiclass' and set 'num_class' if you have more than 2 classes
    'metric': 'binary_logloss',  # Use 'multi_logloss' if multiclass
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'verbose': -1
}

# Train the model
model = lgb.train(params, train_data, valid_sets=[test_data])


In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]  # Convert probabilities to binary values for binary classification

# Evaluate accuracy
accuracy = accuracy_score(Y_test, y_pred_binary)
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, model.predict(X_test)) 
roc_auc = auc(fpr, tpr)
# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Delirium Classification')
plt.legend()
plt.show()

In [ ]:
# Get feature importance
importance = model.feature_importance()
feature_names = X_train.columns

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importance})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['feature'], feature_importance_df['importance'])
plt.xlabel("Feature Importance")
plt.title("LightGBM Feature Importance")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Assuming 'model' is your trained LightGBM model and 'X_train' is your training data

# Create SHAP TreeExplainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)


# Detailed SHAP summary plot
shap.summary_plot(shap_values, X_train, show=False)
plt.title("LightGBM SHAP Summary Plot")
plt.show()


In [ ]:
# Step 4: Create and Train XGBoost Classifier
train_data = xgb.DMatrix(X_train, label=Y_train)  # Convert training data to DMatrix
test_data = xgb.DMatrix(X_test, label=Y_test)     # Convert test data to DMatrix

# Set parameters
params = {
    'objective': 'binary:logistic',  # For binary classification
    'eval_metric': 'logloss',        # Use 'mlogloss' for multiclass
    'eta': 0.1,                      # Learning rate
    'max_depth': 6
}

# Train the model
model = xgb.train(params, train_data, num_boost_round=100, evals=[(test_data, 'test')],
                  early_stopping_rounds=10, verbose_eval=False)


In [ ]:
# Convert X_test to DMatrix for predictions
y_pred_prob = model.predict(test_data)  # Use the DMatrix format here
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]  # Convert probabilities to binary values

# Evaluate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, model.predict(test_data)) 
roc_auc = auc(fpr, tpr)
# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Delirium Classification')
plt.legend()
plt.show()

In [ ]:
# Assuming 'model' is your trained XGBoost model and 'X_train' is your training data

# Create SHAP TreeExplainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

# Detailed SHAP summary plot
shap.summary_plot(shap_values, X_train, show=False)
plt.title("XGBoost SHAP Summary Plot")
plt.show()



In [ ]:
# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)

# Train the model
rf_model.fit(X_train, Y_train)


In [ ]:
# Make predictions on the test set
y_pred = rf_model.predict(X_test)
y_pred2 = rf_model.predict(X_train)

# Evaluate accuracy
accuracy = accuracy_score(Y_test, y_pred)
accuracy = accuracy_score(Y_train, y_pred2)
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, rf_model.predict(X_test)) 
roc_auc = auc(fpr, tpr)
# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Delirium Classification')
plt.legend()
plt.show()

In [ ]:
rf_model.feature_importances_

In [ ]:
# Create SHAP TreeExplainer
explainer = shap.TreeExplainer(rf_model)

# Calculate SHAP values
shap_values = explainer.shap_values(X_train)

In [ ]:
shap_values[0]

In [ ]:
len(shap_values[1])

In [ ]:
X_train.columns

In [ ]:
# Detailed SHAP summary plot
shap.summary_plot(shap_values[-1], X_train, show=False)  # Use shap_values[1] for the positive class
plt.title("Random Forest SHAP Summary Plot")
plt.show()